In [3]:
import cv2 as cv
import numpy as np
import math
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
"""
@file hough_lines.py
@brief This program demonstrates line finding with the Hough transform
"""
import sys
import math
import cv2 as cv
import numpy as np


lines_= []
indices = []

b = 0
a = 0
c =0
def main(argv):
    global indices,a,b,c
    default_file = 'hough_3.png' #enter outputs from DeepNet predictions here
    filename = argv[0] if len(argv) > 0 else default_file
    # Loads an image
    src = cv.imread(default_file, cv.IMREAD_GRAYSCALE)
    # Check if image is loaded fine
    if src is None:
        print ('Error opening image!')
        print ('Usage: hough_lines.py [image_name -- default ' + default_file + '] \n')
        return -1
    
    
    dst = cv.Canny(src, 50, 200, None, 3)
    
    # Copy edges to the images that will display the results in BGR
    cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
    cdstP = np.copy(cdst)
    
    lines = cv.HoughLines(dst, 1, np.pi / 180, 150, None, 0, 0)
    
    if lines is not None:
        
        for i in range(0, len(lines)):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
            pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
            cv.line(cdst, pt1, pt2, (0,0,255), 1, cv.LINE_AA)
            
    
    
    linesP = cv.HoughLinesP(dst, 1, np.pi / 180, 50, None, 50, 10)
    
    
    if linesP is not None:
        #cluster and index finding
        print(len(linesP))
        slopes = np.asarray([(linesP[i][0][3]-linesP[i][0][1])/(linesP[i][0][2]-linesP[i][0][0]) for i in range(0, len(linesP))])
        slopes_indeces = np.where((abs(slopes)>.1763)  & (abs(slopes)!=float("inf")))
        
        linesP = linesP[slopes_indeces]
        slopes = slopes[slopes_indeces]  
        
        a = linesP
         
        X = np.asarray(slopes).reshape(-1,1)
        kmeans = KMeans(n_clusters=4, random_state=0).fit(X) 
        last_4_indeces = np.asarray(pd.Series(kmeans.labels_).drop_duplicates().index)
        linesP = linesP[last_4_indeces]
        
        b = pd.Series(kmeans.labels_) 
        
        clust_cent ={}
        for  i in list(pd.Series(b).drop_duplicates()):
            clust_cent[i] = []
        for i,j in zip(a,b):
            clust_cent[j].append(list(i[0]))
    
        mean_line = []
        mean_line = [np.mean(clust_cent[i],axis=0) for i in clust_cent.keys()]
        #mean_line
        
        
        
        for i in range(0, len(linesP)):
        #for i in range(0,len(mean_line)):
            l = linesP[i][0]
            #l = list(mean_line[i])
            cv.line(cdstP, (int(l[0]), int(l[1])), (int(l[2]), int(l[3])), (0,0,255), 3, cv.LINE_AA)
            slope = (l[3]-l[1])/(l[2]-l[0]) 
            #print(slope)
            #lines_.append(slope)
                
    
    cv.imshow("Source", src)
    cv.imshow("Detected Lines (in red) - Standard Hough Line Transform", cdst)
    cv.imshow("Detected Lines (in red) - Probabilistic Line Transform", cdstP)
    cv.imwrite('mean_hough_lines.jpg',cdstP)
    
    cv.waitKey(0)
    cv.destroyAllWindows()
    return 0
    
if __name__ == "__main__":
    main(sys.argv[1:])

22


C:\Users\ayush\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in int_scalars


0.3099374720850522